In [9]:
import matplotlib.pyplot as plt
   
from numpy import argmax
import keras
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from google.colab import drive
drive.mount('/content/drive')

InputFile ='/content/drive/MyDrive/Dataset/AIS_EURUSD_1-5-15-30+.xlsx'
Model_Path ='/content/drive/MyDrive/Dataset/8_18_2022_4_LastModel_97P'
OutputFile ='/content/drive/MyDrive/Dataset/Predicted_Trend'+datetime.now().strftime("%d_%m_%Y_%H_%M")+'.xlsx'

def find_MaxtimeStep(df):
    # x = df.loc[df['INDEX'].notnull()].index.tolist()
    # max_distance =0 
    # for i in range (0,len(x)-1):
    #     if x[i+1]-x[i] >max_distance:
    #         max_distance = x[i+1]-x[i]
    return 700
def get_fixedSizeCandle(candles,md):
    candles_array = np.array(candles)
    Fixed_MaxSizeCandles = np.zeros((md,candles_array.shape[1]),dtype=float)
    Fixed_MaxSizeCandles[md-candles_array.shape[0]:,:candles_array.shape[1]] =candles_array
    return Fixed_MaxSizeCandles

def PrepareTestData(df,Max_TimeStep):   
    x=list()
    y=list()
    i=0
    candles = list()
    while i<len(df):
        candle =list()
        candle = np.zeros(4)
        if not(pd.isnull(df.at[i,'INDEX'])):
            y.append(df.at[i,'INDEX'])
            if len(candles)>0:
                    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
            candles = list()
        else:
            for j in range (0,4):
                candle[j] = df.iloc[i,j+2] -df.iloc[i-1,j+2]
                #candle[j] = df.iloc[i,j+2]
        candles.append(candle)
        i = i+1
    x.append(get_fixedSizeCandle(candles,Max_TimeStep))
    return np.array(x),np.array(y)

excel_test_data =  pd.read_excel(InputFile)
df1 = pd.DataFrame(excel_test_data, columns=['DATE', 'TIME', 'OPEN','HIGH','LOW','CLOSE','TREND','INDEX'])
Max_TimeStep = find_MaxtimeStep(df1)
testX,textY = PrepareTestData(df1,Max_TimeStep)
scaler = StandardScaler()
for i in range(0,testX.shape[0]):
   testX[i]= scaler.fit_transform(testX[i])

model = keras.models.load_model(Model_Path)
testPredict = model.predict(testX)
print(testPredict)
#df1[['BreakDown','BreakUp','Channel','Down','Side','Up']] = pd.DataFrame(testPredict)

trendlist = df1.loc[df1['INDEX'].notnull()].index.tolist()
answer = ["" for i in range(len(df1))]
BreakDown = ["" for i in range(len(df1))]
BreakUp = ["" for i in range(len(df1))]
Channel = ["" for i in range(len(df1))]
Down = ["" for i in range(len(df1))]
Side = ["" for i in range(len(df1))]
Up = ["" for i in range(len(df1))]
key=['Down Break','Up Break','Channel','Down','Side','Up']
for i in range(0,len(trendlist)):
  answer[trendlist[i]] =key[argmax(testPredict[i, :])]
  BreakDown[trendlist[i]] =testPredict[i, 0]
  BreakUp[trendlist[i]] =testPredict[i, 1]
  Channel[trendlist[i]] =testPredict[i, 2]
  Down[trendlist[i]] =testPredict[i, 3]
  Side[trendlist[i]] =testPredict[i, 4]
  Up[trendlist[i]] =testPredict[i, 5]
df1['BreakDown'] = BreakDown
df1['BreakUp'] = BreakUp
df1['Channel'] = Channel
df1['Down'] = Down
df1['Side'] = Side
df1['Up'] = Up
df1['PredictedTrend'] = answer
df1.to_excel(OutputFile)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[[8.15456390e-01 3.79925905e-06 1.77622259e-01 3.81640893e-05
  8.71900047e-05 6.79216534e-03]
 [3.22855003e-02 1.49182172e-03 3.90732676e-01 1.79207345e-05
  5.78746141e-04 5.74893296e-01]
 [2.17394112e-03 3.31362686e-03 4.21626300e-01 1.02995060e-07
  2.09402002e-04 5.72676659e-01]
 [1.52199413e-03 3.37353069e-03 6.50919020e-01 5.72456678e-08
  1.60079260e-04 3.44025314e-01]
 [1.32954190e-03 3.19343805e-03 8.64454687e-01 3.37385764e-08
  1.31566674e-04 1.30890682e-01]
 [6.71158414e-05 3.50280679e-05 5.02766998e-05 9.99846816e-01
  6.94827463e-07 2.95376679e-12]
 [6.42716259e-05 6.19649938e-07 3.23144413e-05 9.99902725e-01
  1.07946740e-07 1.67908840e-12]
 [1.06456944e-04 1.59449610e-05 1.09737201e-04 9.99767244e-01
  5.94698747e-07 7.16060891e-12]
 [7.58771857e-05 3.49458896e-05 8.87067145e-05 9.99799192e-01
  1.32713069e-06 5.17258458e-12]
 [1.26380415e-04